In [ ]:
%matplotlib inline
import re
import os
import numpy as np
import pandas as pd
import seaborn as sns
from requests import get
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# Import Data

In [ ]:
dfList = []
for r, d, f in os.walk('../data/dunnhumby - The Complete Journey CSV/'):
    for file in f:
        if '.csv' in file:
            print(file)
            dfList.append(pd.read_csv(os.path.join(r, file)))

In [ ]:
campaign_desc_df = dfList[0]
campaign_table_df = dfList[1]
causal_data_df = dfList[2]
coupon_df = dfList[3]
coupon_redempt_df = dfList[4]
hh_demographic_df = dfList[5]
product_df = dfList[6]
transaction_data_df = dfList[7]

# Observing Data

In [ ]:
campaign_desc_df.head(2)

In [ ]:
campaign_table_df.head(2)

In [ ]:
causal_data_df.head(2)

In [ ]:
coupon_df.head(2)

In [ ]:
coupon_redempt_df.head(2)

In [ ]:
hh_demographic_df.head(2)

This seems to be interesting data about households

## Product Data

In [ ]:
product_df.head(10)

In [ ]:
product_df.groupby('DEPARTMENT')['PRODUCT_ID'].count().plot.bar(rot=90,figsize=(9,5))

Let us look at what kind of grocery items we can find

In [ ]:
product_df[product_df['DEPARTMENT']=='GROCERY'].groupby('COMMODITY_DESC')['PRODUCT_ID'].count().sort_values(ascending=False)

In [ ]:
product_df[(product_df['DEPARTMENT']=='GROCERY') & (product_df['COMMODITY_DESC']=='SOFT DRINKS')]

Let us look at what kind of 'Drug GM' products we have

In [ ]:
product_df[product_df['DEPARTMENT']=='DRUG GM'].groupby('COMMODITY_DESC')['PRODUCT_ID'].count().sort_values(ascending=False)

## Transaction Data

In [ ]:
transaction_data_df.head()

# Question 1: How is purchasing rate related to demographics?

If we want to compare shopping amounts amongst households, we are subject to transaction amounts and the time in which they took place. We can compare the shopping rate amongst households by taking the ratio of the cummulative expenditure of a household and the number of days over which they occured.

In [ ]:
# turn the days of purchase into a list on a single row
days_purchased_each_house=transaction_data_df.groupby(['household_key','DAY','STORE_ID'])['SALES_VALUE']\
.sum()\
.reset_index()\
.set_index('household_key')\
.groupby('household_key')['DAY'].apply(list)
days_purchased_each_house.head()


In [ ]:
#turn the transaction values into a list on a single row
transaction_val_each_house=transaction_data_df.groupby(['household_key','DAY','STORE_ID'])['SALES_VALUE']\
.sum()\
.reset_index()\
.set_index('household_key')\
.groupby('household_key')['SALES_VALUE'].apply(list)
transaction_val_each_house.head()

In [ ]:
#merge the two series on household key
transaction_freq_df = pd.merge(transaction_val_each_house,days_purchased_each_house,\
                               left_on='household_key',right_on='household_key')


In [ ]:
transaction_freq_df.head()
    

In [ ]:
# Compute the purchase rate 
def compute_transaction_rate(household_df):
    
    #Compute cumulative sum of sales value
    cum_trans = sum(household_df['SALES_VALUE'])
    
    #subtract last day from first day
    x = np.array(household_df['DAY'])
    
    if (x[-1]-x[0])==0: #we filter out single transactions
        purchase_rate = np.nan
    else:
        purchase_rate = cum_trans/(x[-1]-x[0])
    
    return purchase_rate

In [ ]:
#get purchase rate series
purchase_rate=transaction_freq_df.apply(compute_transaction_rate,axis=1)

#observe distribution of purchase rate
purchase_rate.hist(bins=200)
ax = plt.gca()
ax.set_title('Purchasing Rate vs. Household')
ax.set_xlabel('Purchasing rate ($/day)')
ax.set_ylabel('Frequency')


In [ ]:
#Find outliers
purchase_rate.sort_values(ascending=False)

In [ ]:
#merge with demographic df
purchase_rate.rename('purchase_rate',inplace=True)
hh_demographic_df = hh_demographic_df.join(purchase_rate,on='household_key')

In [ ]:
hh_demographic_df.\
groupby('HOUSEHOLD_SIZE_DESC')['purchase_rate'].agg(['mean','std'])

No relevant correlation with household size

In [ ]:
#income description vs. purchase rate
idesc_purchrate = hh_demographic_df\
.groupby(['INCOME_DESC','purchase_rate'])['household_key']\
.count()\
.reset_index().drop(columns='household_key').sort_values(by='purchase_rate',ascending=False)
idesc_purchrate.head(10)

Let's look at a box-plot instead, since there can be outliers shifting the means

In [ ]:
#specify order of boxes in boxplot
order = ['250K+','200-249K','175-199K','150-174K','125-149K','100-124K', '75-99K','50-74K','35-49K','25-34K','15-24K']
order = [o for o in reversed(order)]
#do a box plot
p1=sns.stripplot(x=idesc_purchrate.INCOME_DESC, y=idesc_purchrate.purchase_rate, data=idesc_purchrate,order=order,size=2,color='.3')
p1.set_xticklabels(p1.get_xticklabels(),rotation=45)
p1 = sns.boxplot(x=idesc_purchrate.INCOME_DESC, y=idesc_purchrate.purchase_rate, data=idesc_purchrate,order=order)
p1.set_ylabel('Purchase Rate ($/day)')
p1.set_xlabel('Income Description')

In [ ]:
hh_demographic_df.groupby('AGE_DESC')['purchase_rate'].agg(['mean','std']).sort_values(by='mean',ascending=False)

It seems that the strongest correlation exists with income groups. But, we don't know if they are buying the same items. This will be our next question

# What kind of items are bought by these demographic groups? 

In [ ]:
# merge transaction df with product df
transaction_product_id_df = transaction_data_df[['household_key','PRODUCT_ID','QUANTITY','SALES_VALUE','DAY']]\
.merge(product_df[['PRODUCT_ID','DEPARTMENT','COMMODITY_DESC','SUB_COMMODITY_DESC','CURR_SIZE_OF_PRODUCT']],on=['PRODUCT_ID'],how='left')

In [ ]:
transaction_product_id_df[transaction_product_id_df['DEPARTMENT']=='GROCERY'].\ # filter grocery items
groupby(['household_key','COMMODITY_DESC'])[['SALES_VALUE','QUANTITY']].\ #groupby household key and commodity type
sum().sort_values(['household_key','SALES_VALUE'],ascending=False).reset_index().\ #sum sales value and sort per household
groupby('household_key').first().reset_index().\ # take top item per household
groupby('COMMODITY_DESC')['household_key'].count().\# group by commodity type and count how many houses share top item
sort_values(ascending=False).head(20).plot.bar(rot=90) #plot top 20 commodity types
ax = plt.gca()
ax.set_title('Item of Highest Sales per Household')
ax.set_ylabel('# households')

We can try to compare the mass of the products added most of the products are measured in ounces, pounds or mL

In [ ]:
def get_solid_weight(grocery_prod_df):
    "get size in oz"
    size_input = grocery_prod_df['CURR_SIZE_OF_PRODUCT']
    # check for OZ
    rex = re.search('[.0-9]{1,5}\s{0,1}OZ', size_input)
    if (rex!=None):
        rex = re.search('[.0-9]{1,5}',rex.group())
        size = float(rex.group())
    else:
        rex = re.search('[.0-9]{1,5}\s{0,1}LB',size_input)
        
        if rex != None:
            rex = re.search('[.0-9]{1,5}',rex.group())
            size = float(rex.group())*16
        else:
            size = np.nan
    return size
        
    
    
    

In [ ]:
product_df.set_index('PRODUCT_ID',inplace=True)

In [ ]:

product_weight = product_df[product_df['DEPARTMENT']=='GROCERY'].apply(get_solid_weight,axis=1).sort_values(ascending=False).rename('weight_in_oz')

In [ ]:
grocery_solid_prods = product_df.join(product_weight).dropna(subset=['weight_in_oz'],how='any')

In [ ]:
grocery_solid_prods

In [ ]:
transaction_product_id_df = transaction_data_df[['household_key','PRODUCT_ID','QUANTITY','SALES_VALUE','DAY']]\
.merge(grocery_solid_prods[['COMMODITY_DESC','SUB_COMMODITY_DESC','weight_in_oz']],\
       how='right',left_on='PRODUCT_ID',right_index=True)

In [ ]:
transaction_product_id_df

In [ ]:
transaction_product_id_df['tot_weight']=(transaction_product_id_df['weight_in_oz']*transaction_product_id_df['QUANTITY'])

In [ ]:
transaction_product_id_df.head()

In [ ]:
transaction_product_id_df[['sales_sum','weight_sum']]=transaction_product_id_df\
.groupby(['household_key','SUB_COMMODITY_DESC'])['SALES_VALUE','tot_weight']\
.transform('sum')
transaction_product_id_df.head()

In [ ]:
transaction_product_id_df.sort_values('weight_sum',ascending=False).groupby('household_key').head(1)\
.groupby('COMMODITY_DESC')['PRODUCT_ID'].count()\
.sort_values(ascending=False)\
.head(15).plot.bar(rot=90,figsize=(9,5))
ax = plt.gca()
ax.set_title('')